In [158]:
import numpy as np
import pandas as pd
import math
import re
from shapely import Polygon

In [159]:
#FILE_NAME = "test.txt"
#FILE_NAME = "test2.txt"
FILE_NAME = "input.txt"
DIAG=False
DIAG2 = False

In [160]:
MODULES = {}
TYPE_FLIPFLOP = "flipflop"
TYPE_BROADCAST = "broadcast"
TYPE_CONJUNCT = "conjuction"
TYPE_BUTTON = "button"

NAME_BROADCASTER = "broadcaster"
NAME_BUTTON = "button"

PULSE_LOW = -1
PULSE_HIGH = 1

STATE_OFF = -2
STATE_ON = 2

State means:
    - flipflop - False means off
    - conjunction - list of connected modules

In [161]:
def get_module_inst(mod_name,mod_type,descendants):
    desc_list = [x.strip() for x in descendants]
    return {"name":mod_name,"type":mod_type,"descend": desc_list,"state_flip":STATE_OFF,"state_conj":{}}


In [162]:
def get_module_inst_button():
    return get_module_inst(NAME_BUTTON,TYPE_BUTTON,[NAME_BROADCASTER])

In [163]:
def buildModulesFromLines(lines):
    
    cur_x = 0
    cur_y = 0
    
    res = []
    
    for line in lines:
        if line[-1]=="\n": line = line[:-1]
        if len(line)>0:
            module_str,downstream = line.split("->")
            
            module_str =  module_str.strip()
             
            
            if module_str[0] == "%":
                mod_type=TYPE_FLIPFLOP
                module_str = module_str[1:]
            elif module_str[0] == "&":
                mod_type = TYPE_CONJUNCT
                module_str = module_str[1:]
            elif module_str.strip()== NAME_BROADCASTER:
                mod_type = TYPE_BROADCAST
            else:
                print ("HOLY SHIT SHOULDN'T HAPPEN!")
            
            module_inst = get_module_inst(module_str,mod_type,downstream.split(","))

    
            MODULES[module_str] = module_inst
    
    # parse all modules to figure out precedecessors of conjunction modules
    
    
    reverse_module_map = []
    for mod_name, mod_obj in MODULES.items():
        desc_list = mod_obj["descend"]
        for x in desc_list:
            reverse_module_map.append((x,mod_name))
    
    global reverse_map
    reverse_map = {}
    print ("Reverse module")
    for x in MODULES:
        reverse_map[x]=[z for (y,z) in reverse_module_map if y == x]
    #    print(">>",x,reverse_map[x])
    
    global to_check 
    to_check = []
    for x in ["ff","th","hb","tj"]:
        to_check += reverse_map[x]
    
    #for x in reverse_module_map:
    #    print(">>",x)
    
    for mod_name, mod_obj in MODULES.items():
        mod_type = mod_obj["type"]
        if mod_type == TYPE_CONJUNCT:
            predec_list = {y:PULSE_LOW for x,y in reverse_module_map if x == mod_name}
            mod_obj["state_conj"] = predec_list
            MODULES[mod_name]=mod_obj
            
    MODULES[NAME_BUTTON]=get_module_inst_button()
    
    return MODULES
    
            

In [164]:
def process_single_pulse(orig_name,dest_name,pulse_type,all_modules):
    if dest_name == GLOB_CHECK:
        if pulse_type == PULSE_LOW and obj_dest[dest_name]["state_flip"] == STATE_OFF:
            raise Exception("Found")
    elif dest_name not in all_modules:
        return []
    
    obj_dest = all_modules[dest_name]
    obj_dest_type = obj_dest["type"]
    if DIAG2: print("PULSE",orig_name,pulse_type,dest_name,"---",obj_dest_type)
    
    #to_send 
    if obj_dest_type == TYPE_BROADCAST:
        # send same pulse to all
        to_send = pulse_type
    elif obj_dest_type == TYPE_FLIPFLOP:
        if pulse_type == PULSE_HIGH:
            return []
        else:
            obj_dest["state_flip"] = - obj_dest["state_flip"]
            if obj_dest["state_flip"] == STATE_ON:
                # send HIGH PULSE to all
                to_send = PULSE_HIGH
            else:
                to_send = PULSE_LOW
    elif obj_dest_type == TYPE_CONJUNCT:
        obj_dest["state_conj"][orig_name] = pulse_type
        if DIAG: print(">>",obj_dest["state_conj"])
        if all([x==PULSE_HIGH for x in obj_dest["state_conj"].values()]):
            to_send = PULSE_LOW
        else:
               to_send = PULSE_HIGH
        if DIAG: print("<<",obj_dest["state_conj"])
    else:
        print("INCORRECT TARGET MODULE!!!")
        return []
    
    new_pulses = [(dest_name,x,to_send) for x in obj_dest["descend"]]  
    if DIAG: print("New pulses: ",new_pulses)
    return new_pulses

In [165]:
def process_pulses(modules):
    low_count = 0
    high_count = 0 
    pulse_list = [(NAME_BUTTON,NAME_BROADCASTER,PULSE_LOW)]
    while len(pulse_list)>0:
        pulse = pulse_list[0]
        if pulse[2] < 0:
            low_count+=1
        else:
            high_count +=1
        pulse_list = pulse_list[1:]
    
        pulse_list+= process_single_pulse(*pulse,modules)
        if DIAG: print("----\n",pulse_list)
    return (low_count,high_count,modules)

In [166]:
# OK
def parseFile(fileName):   
    with open(fileName) as file:
        lines =  file.readlines()    
   
    MODULES = buildModulesFromLines(lines)
    
   
    
    
    return MODULES
    

In [173]:
modules = parseFile(FILE_NAME)
if DIAG: print(modules)
#PULSES_COUNT = 10000000
PULSES_COUNT = 10000

to_check_res = {}
global GLOB_CHECK

for x in to_check:
    to_check_res[x]=-1

print(">>",to_check)

for x in to_check:
    GLOB_CHECK=x
    counter=0
    while counter < PULSES_COUNT:    
        try:
            low_c,high_c,modules=process_pulses(modules)
        except:
            #print("Element {}, counter {}".format(x,counter))
            to_check_res[x]=counter+1
            break
        counter+=1
print("Result: \n",min(to_check_res.values()))
      
print(to_check_res)
print(math.lcm(*to_check_res.values()))

Reverse module
>> ['vd', 'ld', 'bn', 'fq', 'rs', 'zz', 'sq', 'mb', 'jj', 'bv', 'rk', 'xt', 'rc', 'jh', 'mf', 'st', 'fn', 'dx', 'mv', 'rl', 'km', 'tp', 'ch', 'xp', 'fz', 'fx', 'qm', 'rf', 'dg', 'lp', 'lq', 'js', 'kc']
Result: 
 1
{'vd': 256, 'ld': 1024, 'bn': 64, 'fq': 2048, 'rs': 4, 'zz': 1, 'sq': 8, 'mb': 512, 'jj': 51, 'bv': 2, 'rk': 147, 'xt': 83, 'rc': 121, 'jh': 422, 'mf': 16, 'st': 1024, 'fn': 1, 'dx': 344, 'mv': 8, 'rl': 512, 'km': 128, 'tp': 1, 'ch': 4, 'xp': 32, 'fz': 256, 'fx': 1024, 'qm': 7, 'rf': 640, 'dg': 512, 'lp': 1, 'lq': 2048, 'js': 16, 'kc': 64}
2331742488176640


2331742488176640 is too high
241528184647003

1213418

In [168]:
#res == 32000000
res == 11687500

False

In [169]:
x="rx"
reverse_map

{'kl': ['ff'],
 'vd': ['lg'],
 'dx': ['fz'],
 'jj': ['qr'],
 'ld': ['mb'],
 'bn': ['zp'],
 'mv': ['ch'],
 'mx': ['mv', 'hb'],
 'qm': ['kc'],
 'zd': ['sq', 'ff'],
 'tq': ['lr', 'th'],
 'vm': ['th'],
 'qr': ['mf', 'th'],
 'bv': ['fn'],
 'rf': ['dg'],
 'broadcaster': [],
 'rk': ['xt'],
 'tj': ['qm', 'rf', 'dg', 'lp', 'lq', 'js', 'kc'],
 'dg': ['gz'],
 'xt': ['jj'],
 'fq': ['ld'],
 'gz': ['qm', 'tj'],
 'rl': ['fx'],
 'rc': ['rk'],
 'km': ['xf'],
 'gv': ['tj', 'lp'],
 'lr': ['bv', 'th'],
 'lg': ['bn', 'ff'],
 'jh': ['st'],
 'rs': ['fj'],
 'bt': ['tj', 'js'],
 'mf': ['tq'],
 'xf': ['xp', 'hb'],
 'tp': ['broadcaster', 'hb'],
 'ch': ['sv'],
 'xp': ['mx'],
 'xh': ['tj', 'ct'],
 'fz': ['km'],
 'zp': ['zd', 'ff'],
 'kv': ['hb'],
 'll': ['kl', 'vm', 'kv', 'vb'],
 'zz': ['broadcaster', 'ff'],
 'lp': ['broadcaster', 'tj'],
 'vb': ['tj'],
 'th': ['jj', 'bv', 'rk', 'xt', 'rc', 'jh', 'mf', 'st', 'fn'],
 'sq': ['rs'],
 'st': ['rc'],
 'fx': ['dx'],
 'fj': ['zz', 'ff'],
 'lq': ['rf'],
 'fn': ['broadcaster

In [170]:
#to_check = []
#for x in ["ff","th","hb","tj"]:
#    to_check += reverse_map[x]

for x in to_check:
    print(x,modules[x]["type"])


vd flipflop
ld flipflop
bn flipflop
fq flipflop
rs flipflop
zz flipflop
sq flipflop
mb flipflop
jj flipflop
bv flipflop
rk flipflop
xt flipflop
rc flipflop
jh flipflop
mf flipflop
st flipflop
fn flipflop
dx flipflop
mv flipflop
rl flipflop
km flipflop
tp flipflop
ch flipflop
xp flipflop
fz flipflop
fx flipflop
qm flipflop
rf flipflop
dg flipflop
lp flipflop
lq flipflop
js flipflop
kc flipflop
